# DACON 4 COVID - 2차 제출 - EDA 3)Patient
## 제목 : COVID-19 Data EDA : 시각화로 한눈에 이해하는 COVID-19 (Update ver.2020-04-19)

### 코드설명 :

안녕하세요. 팀 쇠똥구리 입니다.

저희 팀의 1차 제출은 "COVID-19 data EDA : 시각화로 한눈에 이해하는 COVID-19"입니다. 

* EDA란? Exploratory Data Analysis의 줄임말로 탐색적 데이터 분석이라고 합니다. 데이터를 다양한 각도에서 관찰하고 이해하는 과정으로, 한마디로 데이터를 분석하기 전에 그래프나 통계적인 다양한 방법을 통해 자료를 직관적으로 이해하는 과정입니다.

본 대회의 목적인 COVID-19 확산 방지 및 예방을 위한 인사이트 발굴 이전에 EDA 과정이 우선되어야 할 텐데요, 

저희는 본격적인 분석 이전에, 데이터를 전체적으로 살펴보고 개별 값을 이해하며, 현상을 더 잘 바라볼 수 있도록 다양한 각도에서 살펴보는 과정을 선행하였습니다. 

EDA의 과정을 통해 필요한 데이터의 수집을 결정하고 미쳐 발견하진 못한 잠재적 패턴을 발굴하기도 합니다. (궁금하다면 3차 제출을 기대해주세요!)

특히 시각화는 데이터를 한눈에 이해하는데에 큰 도움이 됩니다.

저희의 첫 제출인 EDA는 주어진 개별 데이터들을 key 값의 내용에 따라 크게 3파트로 나누어 

1. Time Related, 2. Region Related, 3. Case + Patient Related 

로 나누어 분석을 진행해 보았습니다.

그럼 재미있게 보시고, 내용이 유익하셨다면 투표 부탁드립니다!

감사합니다.

* 링크 1 - 1. Time Related
* 링크 2 - 2. Region Related,
* 링크 3 - 3. Case + Patient Related 
* 링크 4 - EDA summary

---- 
# 0. Setting

필요한 모듈을 import한 후 dataset(Updated ver.2020.04.20)을 읽어들입니다.

In [60]:
import numpy as np 
import pandas as pd
import os
import glob
import re
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import animation, rc
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import plotly.express as px
import folium
from folium import plugins
import branca
import json
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.io as pio
import matplotlib.pylab as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

pio.renderers.default = "notebook_connected"

In [61]:
files = glob.glob('dataset/*.csv')
data = []
for i in files :
    i = re.sub("dataset\\\\", "", i)
    data.append(re.sub(".csv","",i))
for i in range(len(files)) :
    globals()[data[i]] = pd.read_csv(files[i])
data

['Case',
 'PatientInfo',
 'PatientRoute',
 'Policy',
 'Region',
 'SearchTrend',
 'SeoulFloating',
 'Time',
 'TimeAge',
 'TimeGender',
 'TimeProvince',
 'Weather']

- - -

# 3. Case + Patient Related

### 1) Patient : 확진자 경로

주어진 PatientRoute 데이터를 가지고 확진자의 경로를 보기 쉽게 지도 위에 애니메이션으로 나타냈습니다.

#### 경로의 색깔 : 확진자가 접촉한 사람 수에 따라서

#### 접촉자 0 명 : 초록색

#### 접촉자 1~10 명 : 파란색

#### 접촉자 11~100명 : 노란색

#### 접촉자 100 초과 : 빨간색

#### 접촉자 수가 최다인 사람 : 검정색

In [89]:
contnum = PatientInfo.iloc[:,[0,12]]
clr = []
for i in range(len(contnum)) :
    if contnum.contact_number[i] == 0 :
        clr.append("green")
    elif contnum.contact_number[i] <= 10 :
        clr.append("blue")
    elif contnum.contact_number[i] <= 100 :
        clr.append("yellow")
    elif contnum.contact_number[i] == max(contnum.contact_number) :
        clr.append("black")
    else : 
        clr.append("red")
contnum["color"] = clr
wcontnum = pd.merge(PatientRoute,contnum,
        how ='left',
        on = 'patient_id').fillna(0)
route_geo = gpd.GeoDataFrame(wcontnum,geometry=gpd.points_from_xy(PatientRoute.longitude,PatientRoute.latitude))

linerange = route_geo[route_geo['patient_id'] == route_geo['patient_id'].shift(1)].index
linerange = linerange.drop(linerange[-1])
lines = [ {
                "coordinates": [
                 [route_geo["longitude"][i] , route_geo["latitude"][i] ],
                 [route_geo["longitude"][i+1] , route_geo["latitude"][i+1],]
                   ],
                "dates": [
                 route_geo["date"][i],
                 route_geo["date"][i+1]
                   ],
                "color": route_geo["color"][i]
                     } for i in linerange ] 
features = [
     {
         'type': 'Feature',
         'geometry': {
            'type': 'LineString',
            'coordinates': line['coordinates'],
                },
         'properties': {
            'times': line['dates'],
            'style': {
                'color' : line['color']
            },
            'icon' : 'circle',
            'iconstyle':{
                    'fillOpacity': 0.8,
                    'stroke': 'true',
                    'radius': 3
                }
                }
        }
        for line in lines
    ]


m = folium.Map(
  location=[36.5053542,127.7043419],
  zoom_start=8
)
plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='PT6H', add_last_point=True).add_to(m)
m.save('with_color.html')
m

### 2) 시간에 따른 infection case 별 확진자 수

In [90]:
inf_case = PatientInfo.infection_case.unique()
for i in range(len(inf_case)) :
    globals()['case'+ str(i)] = PatientInfo[PatientInfo.infection_case == inf_case[i]]
    globals()['case'+str(i)] = pd.DataFrame(globals()['case'+ str(i)].groupby('confirmed_date').count().cumsum().patient_id).reset_index()
    globals()['case'+str(i)].columns = ['confirmed_date','case'+str(i)]   

merged = case0
for i in [x for x in range(len(inf_case)) if x != 0] :
    merged = pd.merge(merged , globals()['case'+str(i)],how= 'outer',on='confirmed_date')
merged = merged.sort_values(by = 'confirmed_date').reset_index()
merged = merged.drop('index',axis=1)
merged.columns = np.insert(inf_case,0,'confirmed_date')
merged = merged.fillna(method='pad')
merged = merged.fillna(0)
merged = merged.set_index('confirmed_date')
fig = merged.iplot(asFigure=True, kind='scatter')
fig.show()

----------
### End of Document